In [1]:
%matplotlib inline

from pathlib import Path
import os

import geopandas as gp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

DATA_EXT = (Path(os.getcwd()) / os.pardir / 'data' / 'external').resolve()

In [2]:
import sys
sys.path.append(os.path.join(os.getcwd(), os.pardir, 'src'))

%load_ext autoreload

%autoreload 2
from visualization.visualize import *

# DEPRECATED. All data from DHS geo datasets was `NaN` for these series, so we have to do that recalc ourselves


## Import DHS shapefile data:

DHS provides some tooling for downloading shapefiles in bulk with geographic estimes here (login required):
https://spatialdata.dhsprogram.com/data/#/multiple/countries/indicators/download


For this dataset, we selected the following countries (all with subnational available)

 - AF
 - AL
 - AO
 - AM
 - AZ
 - BD
 - BJ
 - BO
 - BT
 - BR
 - BF
 - BU
 - KH
 - CM
 - CF
 - TD
 - CO
 - KM
 - CG
 - CD
 - CI
 - DR
 - EC
 - EG
 - ES
 - ER
 - ET
 - GA
 - GH
 - GU
 - GN
 - GY
 - HT
 - HN
 - IA
 - ID
 - JO
 - KK
 - KE
 - KY
 - LS
 - LB
 - MD
 - MW
 - MV
 - ML
 - MR
 - MX
 - MB
 - MA
 - MZ
 - MM
 - NM
 - NP
 - NC
 - NI
 - NG
 - PK
 - PY
 - PE
 - PH
 - RW
 - ST
 - SN
 - SL
 - ZA
 - LK
 - SD
 - SZ
 - TJ
 - TZ
 - TH
 - GM
 - TL
 - TG
 - TT
 - TN
 - TR
 - TM
 - UG
 - UA
 - UZ
 - VN
 - YE
 - ZM
 - ZW

And the following indicators.

 - Crude birth rate (FE_FRTR_W_CBR)
 - Neonatal mortality rate (CM_ECMR_C_NNR)
 - Postneonatal mortality rate (CM_ECMR_C_PNR)
 - Infant mortality rate (CM_ECMR_C_IMR)
 - BCG vaccination received in first year (CH_VAC1_C_BCG)
 - DPT1 vaccination received in first year (CH_VAC1_C_DP1)
 - DPT2 vaccination received in first year (CH_VAC1_C_DP2)
 - DPT3 vaccination received in first year (CH_VAC1_C_DP3)
 - Polio0 vaccination received in first year (CH_VAC1_C_OP0)
 - Polio1 vaccination received in first year (CH_VAC1_C_OP1)
 - Polio2 vaccination received in first year (CH_VAC1_C_OP2)
 - Polio3 vaccination received in first year (CH_VAC1_C_OP3)
 - Measles vaccination received in first year (CH_VAC1_C_MSL)
 - Received all 8 basic vaccinations in first year (CH_VAC1_C_BAS)
 - Received no vaccinations in first year (CH_VAC1_C_NON)
 - Percentage showing a vaccination card (CH_VAC1_C_VCD)
 - Number of children one to four years of age (CH_VAC1_C_NUM)
 - Number of children one to four years of age (unweighted) (CH_VAC1_C_UNW)
 - Percentage of population age 0-4 (HC_AGEG_P_004)
 - Median date of interview (SV_INTV_H_YRI)


In [8]:
gdf = gp.read_file(str(Path("../data/external/DHS/sdr_subnational_data_2018-03-04/shps/sdr_subnational_data.shp")))
gdf.head()

,ISO,FIPS,DHSCC,SVYTYPE,SVYYEAR,CNTRYNAMEE,CNTRYNAMEF,CNTRYNAMES,DHSREGEN,DHSREGFR,...,CHVAC1COP3,CHVAC1CMSL,CHVAC1CBAS,CHVAC1CNON,CHVAC1CVCD,CHVAC1CNUM,CHVAC1CUNW,HCAGEGP004,SVINTVHYRI,geometry
0,AL,AL,AL,DHS,2008.0,Albania,,,Coastal,,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,(POLYGON ((19.98586082388749 39.68539237988307...
1,AL,AL,AL,DHS,2008.0,Albania,,,Central,,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,(POLYGON ((19.37545204159284 41.84852790931262...
2,AL,AL,AL,DHS,2008.0,Albania,,,Mountain,,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,"POLYGON ((20.06852531369549 42.55660057181291,..."
3,AL,AL,AL,DHS,2008.0,Albania,,,Tirana,,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,"POLYGON ((19.77230262674408 41.36361503562699,..."
4,AZ,AJ,AZ,DHS,2006.0,Azerbaijan,,,Baku,,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,(POLYGON ((49.45011138923508 39.80466842627226...


In [4]:
# we never use the shapes, so toss them
gdf.drop('geometry', axis=1, inplace=True)

# 9999 is a NaN, drop rows where all of the vaxes are nan
gdf.replace({9999.0: np.nan}, inplace=True)
vax_cols = [c for c in gdf.columns if c.startswith('CHVAC')]
non_nan_mask = pd.notnull(gdf[vax_cols]).all(axis=1)
gdf = gdf[non_nan_mask]

In [9]:
# gdf.replace({9999.0: np.nan}, inplace=True)
vax_cols = [c for c in gdf.columns if c.startswith('CHVAC')]
# non_nan_mask = pd.notnull(gdf[vax_cols]).any(axis=1)

for c in vax_cols:
    print(gdf[c].value_counts())

9999.0    856
Name: CHVAC1CBCG, dtype: int64
9999.0    856
Name: CHVAC1CDP1, dtype: int64
9999.0    856
Name: CHVAC1CDP2, dtype: int64
9999.0    856
Name: CHVAC1CDP3, dtype: int64
9999.0    856
Name: CHVAC1COP0, dtype: int64
9999.0    856
Name: CHVAC1COP1, dtype: int64
9999.0    856
Name: CHVAC1COP2, dtype: int64
9999.0    856
Name: CHVAC1COP3, dtype: int64
9999.0    856
Name: CHVAC1CMSL, dtype: int64
9999.0    856
Name: CHVAC1CBAS, dtype: int64
9999.0    856
Name: CHVAC1CNON, dtype: int64
9999.0    856
Name: CHVAC1CVCD, dtype: int64
9999.0    856
Name: CHVAC1CNUM, dtype: int64
9999.0    856
Name: CHVAC1CUNW, dtype: int64


### Add alpha-3 rather than alpha-2 iso codes

We use the iso-codes data to add the alpha-3 country codes to the alpha-2 country codes that are present in the shapefiles from DHS.

In [5]:
iso_data = pd.read_csv(Path("../data/external/iso-codes.csv"))

iso_data.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,region-code,sub-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [6]:
gdf = gdf.merge(iso_data[['alpha-2', 'alpha-3']], left_on='ISO', right_on='alpha-2', how='left')
gdf.head()

,ISO,FIPS,DHSCC,SVYTYPE,SVYYEAR,CNTRYNAMEE,CNTRYNAMEF,CNTRYNAMES,DHSREGEN,DHSREGFR,...,CHVAC1CMSL,CHVAC1CBAS,CHVAC1CNON,CHVAC1CVCD,CHVAC1CNUM,CHVAC1CUNW,HCAGEGP004,SVINTVHYRI,alpha-2,alpha-3


## Subset the data

 - Surveys greater than or equal to 2015 since the WHO data is 2016
 - countries that appear in both datasets (WHO and DHS)

In [7]:
# load immunization data
imms = pd.read_csv(Path('../data/interim/calc_cols_added.csv'), index_col=0)
imms.head()

,Iso Code,Country Name,WHO Region,Year,Vaccine Type,Admin1,Admin2,DenomType,Denominator,Numerator,Coverage,recalc_numerator,available_admin,normalized_country,indicator,group,vaccine,timing
0,AFG,Afghanistan,EMRO,2016,BCG,NaN,Aab Band,1.0,1266.0,NaN,51.801245,655.803767,Aab Band,afghanistan,1,BCG,BCG,1st_birth
1,AFG,Afghanistan,EMRO,2016,BCG,NaN,Aab Kamari,1.0,4599.0,NaN,94.674670,4354.088093,Aab Kamari,afghanistan,1,BCG,BCG,1st_birth
2,AFG,Afghanistan,EMRO,2016,BCG,NaN,Aaqcha,1.0,5674.0,NaN,72.116489,4091.889607,Aaqcha,afghanistan,1,BCG,BCG,1st_birth
3,AFG,Afghanistan,EMRO,2016,BCG,NaN,Acheen,1.0,4846.0,NaN,35.203626,1705.967701,Acheen,afghanistan,1,BCG,BCG,1st_birth
4,AFG,Afghanistan,EMRO,2016,BCG,NaN,Adraskan,1.0,3557.0,NaN,81.468327,2897.828383,Adraskan,afghanistan,1,BCG,BCG,1st_birth


In [8]:
VALID_YEAR_START = 2015

# subset survey year
gdf = gdf[gdf.SVYYEAR >= VALID_YEAR_START]

# subset countries
gdf = gdf[gdf['alpha-3'].isin(imms['Iso Code'].unique())]
gdf.head()

,ISO,FIPS,DHSCC,SVYTYPE,SVYYEAR,CNTRYNAMEE,CNTRYNAMEF,CNTRYNAMES,DHSREGEN,DHSREGFR,...,CHVAC1CMSL,CHVAC1CBAS,CHVAC1CNON,CHVAC1CVCD,CHVAC1CNUM,CHVAC1CUNW,HCAGEGP004,SVINTVHYRI,alpha-2,alpha-3


In [9]:
# maps 'Vaccine Type' from WHO data (imms) to vaccines from DHS data
DHS_VACCINES = {
    'DTP1': 'CH_VAC1_C_DP1', 
    'DTP2': 'CH_VAC1_C_DP2', 
    'DTP3': 'CH_VAC1_C_DP3', 
    
    '0PV0': 'CH_VAC1_C_OP0',
    'Pol1': 'CH_VAC1_C_OP0',
    'Pol2': 'CH_VAC1_C_OP0',
    'Pol3': 'CH_VAC1_C_OP0',
    
    'BCG': 'CH_VAC1_C_BCG',
    
    'MCV1': 'CH_VAC1_C_MSL', 
}

In [10]:
# subset immunization data to just the rows we have represented
original_n = imms.shape[0]

imms = imms[(imms['Iso Code'].isin(gdf['alpha-3'].unique()) &
             imms['Vaccine Type'].isin(list(DHS_VACCINES.keys())))]
imms.head()

,Iso Code,Country Name,WHO Region,Year,Vaccine Type,Admin1,Admin2,DenomType,Denominator,Numerator,Coverage,recalc_numerator,available_admin,normalized_country,indicator,group,vaccine,timing


In [11]:
pct = imms.shape[0] / original_n
print(f"{(pct*100):.0f}% Percent of WHO observations appear in relevant DHS surveys")

0% Percent of WHO observations appear in relevant DHS surveys


gdf.head()

In [12]:
gdf.head()

,ISO,FIPS,DHSCC,SVYTYPE,SVYYEAR,CNTRYNAMEE,CNTRYNAMEF,CNTRYNAMES,DHSREGEN,DHSREGFR,...,CHVAC1CMSL,CHVAC1CBAS,CHVAC1CNON,CHVAC1CVCD,CHVAC1CNUM,CHVAC1CUNW,HCAGEGP004,SVINTVHYRI,alpha-2,alpha-3
